# Modelling, Analysis and Benchmarking of a Simple Unbalanced LV Network (with Neutral)

## Introduction

This tutorial will demonstrate how to model an unbalanced LV network with a neutral wire in the
_Roseau Load Flow_ (RLF) solver. We will replicate the network in
[Tutorial-DERHC-1-Earth-Return](./Tutorial-DERHC-1-Earth-Return.ipynb) but with a neutral wire.

Before attempting this tutorial, you should have finished _Tutorial 1_ in this repository for a basic
knowledge of how the RLF solver works. The network is identical to that of _Tutorial-DERHC-1-Earth-Return_
except that a neutral wire is added to the network. The network consists of an MV bus, a MV/LV,
$\Delta$-Y transformer (11kV/0.4kV, 250 kVA) between the source bus and bus A, a 240 mm² 3-phase
line connecting buses A and B, and three 16 mm² single-phase lines connecting bus B with buses C, D
and E each of which serves as a connection point for a house.

<center> <img style="float: middle;" 
          src="../images/LV_Network_With_Neutral.png" 
          alt="Simple LV network"
          width="40%"> 
</center>

**<center> Figure 1. Simple LV Network with a Neutral wire </center>**

The details for the loads in the network are given in the table below.
| Load Name | Phases | Connected bus | Peak Demand (kW) | PF |
| :-------- | :----- | :------------ | :--------------- | :-- |
| Load_1 | 1 | C | 7 | 0.95 |
| Load_2 | 1 | D | 6 | 0.95 |
| Load_3 | 1 | E | 8 | 0.95 |

As this tutorial is very similar to the previous one (using a earth return system), we just define
the networks and compare the results without detailed explanations.


In [ ]:
import dss
import numpy as np
import roseau.load_flow as rlf
from rich.table import Table
from rich.text import Text

In [ ]:
# First create a table to store the results for later comparison between the OpenDSS and RLF results
# If this looks complicated, please ignore it, it is not important for the tutorial
titles = {
    "p_demand": "  Active power demand (P) - {}",
    "q_demand": "  Reactive power demand (Q) - {}",
    "u_load": "  Voltage magnitude - {}'s bus",
    "u_bus_A": "  Voltage magnitude - phase {}",
    "p_lvtr": "  Active power (P) supplied to phase {}",
    "q_lvtr": "  Reactive power (Q) supplied to phase {}",
    "p_lvtr_loss": "  Active power (P) losses",
    "q_lvtr_loss": "  Reactive power (Q) losses",
    "p_line_loss": "  Active power (P)",
    "q_line_loss": "  Reactive power (Q)",
}
results_table = {
    "Loads Powers:": (
        {titles["p_demand"].format(load): {} for load in ("Load_1", "Load_2", "Load_3")}
        | {titles["q_demand"].format(load): {} for load in ("Load_1", "Load_2", "Load_3")}
    ),
    "Loads Voltages:": {titles["u_load"].format(load): {} for load in ("Load_1", "Load_2", "Load_3")},
    "Bus A:": {titles["u_bus_A"].format(phase): {} for phase in (1, 2, 3)},
    "LVTR Transformer:": (
        {titles["p_lvtr"].format(phase): {} for phase in (1, 2, 3)}
        | {titles["q_lvtr"].format(phase): {} for phase in (1, 2, 3)}
    ),
    "Transformer Losses:": {titles["p_lvtr_loss"]: {}, titles["q_lvtr_loss"]: {}},
    "Total Line Losses:": {titles["p_line_loss"]: {}, titles["q_line_loss"]: {}},
}

## Open DSS


### Modelling the Network


In [ ]:
# Set up dss_engine
dss_engine = dss.DSS
DSSText = dss_engine.Text
DSSCircuit = dss_engine.ActiveCircuit
DSSSolution = dss_engine.ActiveCircuit.Solution
ControlQueue = dss_engine.ActiveCircuit.CtrlQueue
dss_engine.AllowForms = 0

# Network Modelling - Creating a Circuit
DSSText.Command = "Clear"
DSSText.Command = "Set DefaultBaseFrequency=50"
DSSText.Command = "New Circuit.Simple_LV_Network"
DSSText.Command = "Edit vsource.source bus1=sourceBus basekv=11 pu=1.0 phases=3"

# Adding the 11kV/0.4kV Transformer
DSSText.Command = "New transformer.LVTR Buses=[sourceBus, A.1.2.3.4] Conns=[delta wye] KVs=[11, 0.4] KVAs=[250 250] %Rs=0.00 xhl=2.5 %loadloss=0 "

# Creating the linecodes
DSSText.Command = "new linecode.240sq nphases=4 R1=0.127 X1=0.072 R0=0.342 X0=0.089 units=km"
DSSText.Command = "new linecode.16sq nphases=2 R1=1.15 X1=0.083 R0=1.2 X0=0.083 units=km"

# Creating the 400V and 230V lines
DSSText.Command = "new line.A_B bus1=A.1.2.3.4 bus2=B.1.2.3.4 length=1 phases=4 units=km linecode=240sq"
DSSText.Command = "new line.B_L1 bus1=B.1.4 bus2=C.1.2 length=0.01 phases=2 units=km linecode=16sq"
DSSText.Command = "new line.B_L2 bus1=B.2.4 bus2=D.1.2 length=0.01 phases=2 units=km linecode=16sq"
DSSText.Command = "new line.B_L3 bus1=B.3.4 bus2=E.1.2 length=0.01 phases=2 units=km linecode=16sq"
# DSSText.Command = "dump linecode.240sq debug"
# DSSText.Command = "dump line.A_B debug"

# Connecting loads to a bus
# Note: I (Ali) couldn't make it work with phases=2 for loads. It will need more digging
# into the OpenDSS model to understand the discrepancy between our phase-neutral load
# definition and theirs. For the time being, keeping phases=1 works fine as long as
# the correct bus phases are used in the load definition
DSSText.Command = "new load.Load_1 bus1=C.1.2 phases=1 kV=(0.4 3 sqrt /) kW=7 pf=0.95 model=1 conn=wye Vminpu=0.85 Vmaxpu=1.20 status=fixed"
DSSText.Command = "new load.Load_2 bus1=D.1.2 phases=1 kV=(0.4 3 sqrt /) kW=6 pf=0.95 model=1 conn=wye Vminpu=0.85 Vmaxpu=1.20 status=fixed"
DSSText.Command = "new load.Load_3 bus1=E.1.2 phases=1 kV=(0.4 3 sqrt /) kW=8 pf=0.95 model=1 conn=wye Vminpu=0.85 Vmaxpu=1.20 status=fixed"

# Set the Control mode and the Voltage bases
DSSText.Command = "set controlmode=static"
DSSText.Command = "set mode=snapshot"
DSSText.Command = "Set VoltageBases=[11 0.4]"
DSSText.Command = "calcvoltagebases"

### Running a Load Flow Simulation


In [ ]:
# Run the power flow simulation
DSSSolution.Solve()
if DSSSolution.Converged:
    print("The Circuit was Successfully Solved")
else:
    raise RuntimeError("DID NOT CONVERGE")

### Accessing the Results


In [ ]:
# Extract active and reactive power of loads
section_title = "Loads Powers:"
table_section = results_table[section_title]
print(section_title)
for active_load in ("Load_1", "Load_2", "Load_3"):
    DSSCircuit.SetActiveElement(f"Load.{active_load}")
    pp, pn = DSSCircuit.ActiveElement.Powers[0::2]
    qp, qn = DSSCircuit.ActiveElement.Powers[1::2]
    active_power_demand = f"{pp + pn:.3f} kW"
    reactive_power_demand = f"{qp + qn:.3f} kVAr"
    p_demand_title = titles["p_demand"].format(active_load)
    table_section[p_demand_title]["OpenDSS"] = active_power_demand
    print(f"{p_demand_title} = {active_power_demand} ")
    q_demand_title = titles["q_demand"].format(active_load)
    table_section[q_demand_title]["OpenDSS"] = reactive_power_demand
    print(f"{q_demand_title} = {reactive_power_demand} ")

# Extract load bus voltages
section_title = "Loads Voltages:"
table_section = results_table[section_title]
print(section_title)
for active_load in ("Load_1", "Load_2", "Load_3"):
    DSSCircuit.SetActiveElement(f"Load.{active_load}")
    bus_name = DSSCircuit.ActiveElement.Properties("bus1").Val
    DSSCircuit.SetActiveBus(bus_name)
    potentials = DSSCircuit.ActiveBus.puVoltages[0::2] + 1j * DSSCircuit.ActiveBus.puVoltages[1::2]
    voltages = potentials[:-1] - potentials[-1]
    (v1,) = np.abs(voltages)
    voltage_title = titles["u_load"].format(active_load)
    voltage_value = f"{v1:.3f} pu"
    table_section[voltage_title]["OpenDSS"] = voltage_value
    print(f"{voltage_title} = {voltage_value}")

# Extract bus A voltages
section_title = "Bus A:"
table_section = results_table.setdefault(section_title, [])
print(section_title)
DSSCircuit.SetActiveBus("A")
potentials = DSSCircuit.ActiveBus.puVoltages[0::2] + 1j * DSSCircuit.ActiveBus.puVoltages[1::2]
voltages = potentials[:-1] - potentials[-1]
for phase, voltage in enumerate(np.abs(voltages), start=1):
    voltage_title = titles["u_bus_A"].format(phase)
    voltage_value = f"{voltage:.3f} pu"
    table_section[voltage_title]["OpenDSS"] = voltage_value
    print(f"{voltage_title} = {voltage_value}")

# Extract transformer active and reactive power as well as losses
section_title = "LVTR Transformer:"
table_section = results_table[section_title]
print(section_title)
DSSCircuit.SetActiveElement("transformer.LVTR")
transformer_p = DSSCircuit.ActiveElement.Powers[0::2]
transformer_q = DSSCircuit.ActiveElement.Powers[1::2]
p_loss, q_loss = DSSCircuit.ActiveElement.Losses * 1e-3
# Indices 0:4 are primary powers, indices 4:8 are secondary powers
for i, phase in enumerate((1, 2, 3), start=4):
    transformer_p_value = f"{abs(transformer_p[i]):.3f} kW"
    transformer_q_value = f"{abs(transformer_q[i]):.3f} kVAr"
    transformer_p_title = titles["p_lvtr"].format(phase)
    table_section[transformer_p_title]["OpenDSS"] = transformer_p_value
    print(f"{transformer_p_title} = {transformer_p_value}")
    transformer_q_title = titles["q_lvtr"].format(phase)
    table_section[transformer_q_title]["OpenDSS"] = transformer_q_value
    print(f"{transformer_q_title} = {transformer_q_value}")
transformer_active_loss = f"{abs(p_loss):.3f} kW"
transformer_reactive_loss = f"{q_loss:.3f} kVAr"
section_title = "Transformer Losses:"
table_section = results_table[section_title]
print(section_title)
p_loss_title = titles["p_lvtr_loss"]
table_section[p_loss_title]["OpenDSS"] = transformer_active_loss
print(f"{p_loss_title} = {transformer_active_loss}")
q_loss_title = titles["q_lvtr_loss"]
table_section[q_loss_title]["OpenDSS"] = transformer_reactive_loss
print(f"{q_loss_title} = {transformer_reactive_loss}")

# Extract line losses
section_title = "Total Line Losses:"
table_section = results_table[section_title]
print(section_title)
lines_p, lines_q = DSSCircuit.LineLosses
total_active_line_loss = f"{lines_p:.3f} kW"
total_reactive_line_loss = f"{lines_q:.3f} kVAr"
line_p_title = titles["p_line_loss"]
table_section[line_p_title]["OpenDSS"] = total_active_line_loss
print(f"{line_p_title} = {total_active_line_loss}")
line_q_title = titles["q_line_loss"]
table_section[line_q_title]["OpenDSS"] = total_reactive_line_loss
print(f"{line_q_title} = {total_reactive_line_loss}")

## Roseau Load Flow


### Modelling the Network


In [ ]:
# Nominal voltages of the MV and LV networks
un_mv = rlf.Q_(11, "kV")
un_lv = rlf.Q_(400, "V")

# Buses
source_bus = rlf.Bus(id="sourceBus", phases="abc", nominal_voltage=un_mv)
bus_a = rlf.Bus(id="A", phases="abcn", nominal_voltage=un_lv)
bus_b = rlf.Bus(id="B", phases="abcn", nominal_voltage=un_lv)
bus_c = rlf.Bus(id="C", phases="an", nominal_voltage=un_lv, min_voltage_level=0.85, max_voltage_level=1.20)
bus_d = rlf.Bus(id="D", phases="bn", nominal_voltage=un_lv, min_voltage_level=0.85, max_voltage_level=1.20)
bus_e = rlf.Bus(id="E", phases="cn", nominal_voltage=un_lv, min_voltage_level=0.85, max_voltage_level=1.20)

# Voltage Source
vs = rlf.VoltageSource(id="source", bus=source_bus, voltages=un_mv)

# TransformerParameters from the OpenDSS parameters
tp = rlf.TransformerParameters.from_open_dss(
    id="LVTR",
    conns=("delta", "wye"),
    kvs=[un_mv, un_lv],
    kvas=(250, 250),
    leadlag="euro",  # <- should be "ansi" (i.e. "Dyn1") but we don't have this in RLF yet
    xhl=2.5,
    rs=0,
    loadloss=0,
    noloadloss=0,
    imag=0,
)

# Transformer
transformer = rlf.Transformer("LVTR", bus1=source_bus, bus2=bus_a, parameters=tp)

# Earth and references of potentials
pref_mv = rlf.PotentialRef(id="pref_mv", element=source_bus)
ground = rlf.Ground(id="ground")
ground.connect(bus=bus_a, phase="n")
pref_lv = rlf.PotentialRef(id="pref_lv", element=ground)

# LineParameters from the OpenDSS linecodes
lp240 = rlf.LineParameters.from_open_dss(
    id="240sq",
    nphases=4,
    r1=rlf.Q_(0.127, "ohm/km"),
    x1=rlf.Q_(0.072, "ohm/km"),
    r0=rlf.Q_(0.342, "ohm/km"),
    x0=rlf.Q_(0.089, "ohm/km"),
    c1=rlf.Q_(3.400, "nF/km"),
    c0=rlf.Q_(1.600, "nF/km"),
    basefreq=rlf.Q_(50, "Hz"),
    normamps=rlf.Q_(400, "A"),
    linetype="OH",
)
lp16 = rlf.LineParameters.from_open_dss(
    id="16sq",
    nphases=2,
    r1=rlf.Q_(1.150, "ohm/km"),
    x1=rlf.Q_(0.083, "ohm/km"),
    r0=rlf.Q_(1.200, "ohm/km"),
    x0=rlf.Q_(0.083, "ohm/km"),
    c1=rlf.Q_(3.400, "nF/km"),
    c0=rlf.Q_(1.600, "nF/km"),
    basefreq=rlf.Q_(50, "Hz"),
    normamps=rlf.Q_(400, "A"),
    linetype="OH",
)

# Lines with neutrals
line_ab = rlf.Line(
    "A_B", bus1=bus_a, bus2=bus_b, phases="abcn", parameters=lp240, length=rlf.Q_(1, "km"), ground=ground
)
line_bc = rlf.Line("B_L1", bus1=bus_b, bus2=bus_c, phases="an", parameters=lp16, length=rlf.Q_(10, "m"), ground=ground)
line_bd = rlf.Line("B_L2", bus1=bus_b, bus2=bus_d, phases="bn", parameters=lp16, length=rlf.Q_(10, "m"), ground=ground)
line_be = rlf.Line("B_L3", bus1=bus_b, bus2=bus_e, phases="cn", parameters=lp16, length=rlf.Q_(10, "m"), ground=ground)


# Loads
def complex_power(p: float, pf: float) -> complex:
    """Convert active power and power factor to complex power."""
    phi = np.arccos(pf)
    q = p * np.tan(phi)
    return p + 1j * q


load1 = rlf.PowerLoad(id="Load_1", bus=bus_c, phases="an", powers=[complex_power(7e3, 0.95)])
load2 = rlf.PowerLoad(id="Load_2", bus=bus_d, phases="bn", powers=[complex_power(6e3, 0.95)])
load3 = rlf.PowerLoad(id="Load_3", bus=bus_e, phases="cn", powers=[complex_power(8e3, 0.95)])

# Electrical Network
en = rlf.ElectricalNetwork.from_element(initial_bus=source_bus)
en

### Running a Load Flow Simulation


In [ ]:
en.solve_load_flow()

### Accessing the Results


In [ ]:
# Active and Reactive Powers of the Loads
section_title = "Loads Powers:"
table_section = results_table[section_title]
print(section_title)
for load in (load1, load2, load3):
    load_powers = load.res_powers.m_as("kVA").sum()
    active_power_demand = f"{load_powers.real:.3f} kW"
    reactive_power_demand = f"{load_powers.imag:.3f} kVAr"
    p_demand_title = titles["p_demand"].format(load.id)
    table_section[p_demand_title]["RLF"] = active_power_demand
    print(f"{p_demand_title} = {active_power_demand}")
    q_demand_title = titles["q_demand"].format(load.id)
    table_section[q_demand_title]["RLF"] = reactive_power_demand
    print(f"{q_demand_title} = {reactive_power_demand}")

# Load bus voltages
section_title = "Loads Voltages:"
table_section = results_table[section_title]
print(section_title)
for load in (load1, load2, load3):
    voltage_value = f"{load.bus.res_voltage_levels.m[0]:.3f} pu"
    voltage_title = titles["u_load"].format(load.id)
    table_section[voltage_title]["RLF"] = voltage_value
    print(f"{voltage_title} = {voltage_value}")

# Bus A voltages
section_title = "Bus A:"
table_section = results_table[section_title]
print(section_title)
for phase, pu_voltage in enumerate(bus_a.res_voltage_levels.m, start=1):
    voltage_value = f"{pu_voltage:.3f} pu"
    voltage_title = titles["u_bus_A"].format(phase)
    table_section[voltage_title]["RLF"] = voltage_value
    print(f"{voltage_title} = {voltage_value}")

# Transformer active and reactive power as well as losses
section_title = "LVTR Transformer:"
table_section = results_table[section_title]
print(section_title)
transformer_powers = transformer.res_powers[1].m_as("kVA")
transformer_power_losses = transformer.res_power_losses.m_as("kVA")
for i, phase in enumerate((1, 2, 3)):
    transformer_p_value = f"{abs(transformer_powers[i].real):.3f} kW"
    transformer_q_value = f"{abs(transformer_powers[i].imag):.3f} kVAr"
    transformer_p_title = titles["p_lvtr"].format(phase)
    table_section[transformer_p_title]["RLF"] = transformer_p_value
    print(f"{transformer_p_title} = {transformer_p_value}")
    transformer_q_title = titles["q_lvtr"].format(phase)
    table_section[transformer_q_title]["RLF"] = transformer_q_value
    print(f"{transformer_q_title} = {transformer_q_value}")
transformer_active_loss = f"{transformer_power_losses.real:.3f} kW"
transformer_reactive_loss = f"{transformer_power_losses.imag:.3f} kVAr"
section_title = "Transformer Losses:"
table_section = results_table[section_title]
print(section_title)
p_loss_title = titles["p_lvtr_loss"]
table_section[p_loss_title]["RLF"] = transformer_active_loss
print(f"{p_loss_title} = {transformer_active_loss}")
q_loss_title = titles["q_lvtr_loss"]
table_section[q_loss_title]["RLF"] = transformer_reactive_loss
print(f"{q_loss_title} = {transformer_reactive_loss}")

# Line losses
section_title = "Total Line Losses:"
table_section = results_table[section_title]
print(section_title)
total_line_loss = en.res_lines["series_losses"].sum() / 1e3  # Convert to kVA
total_active_line_loss = f"{total_line_loss.real:.3f} kW"
total_reactive_line_loss = f"{total_line_loss.imag:.3f} kVAr"
line_p_title = titles["p_line_loss"]
table_section[line_p_title]["RLF"] = total_active_line_loss
print(f"{line_p_title} = {total_active_line_loss}")
line_q_title = titles["q_line_loss"]
table_section[line_q_title]["RLF"] = total_reactive_line_loss
print(f"{line_q_title} = {total_reactive_line_loss}")

## Comparison

Please find below a table that summarizes the results of both simulations.


In [ ]:
table = Table(title="Comparison of results")

table.add_column("Title", justify="left", no_wrap=True)
table.add_column("OpenDSS", header_style="italic", justify="right")
table.add_column("Roseau Load Flow", header_style="italic", justify="right")


for section_title, section in results_table.items():
    table.add_section()
    table.add_row(section_title, style="bold")
    for subsection_title, data in section.items():
        style = "green" if data["RLF"] == data["OpenDSS"] else "red1"
        table.add_row(
            subsection_title,
            Text(data["OpenDSS"], style=style),
            Text(data["RLF"], style=style),
        )
table